In [29]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

In [30]:
import os
llm = Groq(model="llama-3.3-70b-versatile", api_key=os.environ.get("GROQ_API_KEY"))

In [31]:
def calcular_imposto_renda(rendimento:float) -> str:
    """ 
    Calcula o imposto de renda com base no rendimento anual.
    Args:
        rendimento (float): Rendimento anual do contribuinte.
    Returns:
        str: O valor do imposto devido com base no rendimento.
    """
    if rendimento <= 2000:
        return "Isento de imposto de renda."
    elif 2000 < rendimento <= 5000:
        imposto = (rendimento - 2000) * 0.10
        return f"Imposto devido é de: R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}."
    elif 5000 < rendimento <= 10000:
        imposto = (rendimento - 5000) * 0.15 + 300
        return f"Imposto devido é de: R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}."
    else:
        imposto = (rendimento - 10000) * 0.20 + 1050
        return f"Imposto devido é de: R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}."

### Convertendo Função em Ferramenta

In [32]:
from llama_index.core.tools import FunctionTool

In [33]:
ferramenta_imposto_renda = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name="Calcular Imposto de Renda",
    description=(
    "Calcula o imposto de renda com base no rendimento anual do contribuinte."
    "Argumento: redimento (float)."
    "Retorna o valor do imposto devido de acordo com faixas de rendimento."
    )
)

In [34]:
from llama_index.core.agent import FunctionCallingAgentWorker 

In [35]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta_imposto_renda],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

In [36]:
from llama_index.core.agent import AgentRunner

In [37]:
agent_imposto= AgentRunner(agent_worker_imposto)

In [38]:
response = agent_imposto.chat("""
    Qual é o imposto de renda devido por uma pessoa com rendimento anual de R$ 7500?
"""
)

Added user message to memory: 
    Qual é o imposto de renda devido por uma pessoa com rendimento anual de R$ 7500?

=== Calling Function ===
Calling function: Calcular Imposto de Renda with args: {"rendimento": 7500}
=== Function Output ===
Imposto devido é de: R$ 675.00, base em um rendimento de R$ 7500.00.
=== LLM Response ===
O imposto de renda devido por uma pessoa com rendimento anual de R$ 7500 é de R$ 675,00.


In [39]:
import arxiv 

def consulta_artigos(titulo: str) -> str:
    """ Consulta artigos na base de dados arXiv e retorna resultados formatados."""
    busca = arxiv.Search(
        query=titulo,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )

    resultados = [
        f"Título: {artigo.title}\n"
        f"Categoria: {artigo.primary_category}\n"
        f"Link: {artigo.entry_id}\n"
        for artigo in busca.results()
    ]

    return "\n\n".join(resultados)

In [40]:
consulta_artigos_tool = FunctionTool.from_defaults(fn=consulta_artigos)

In [41]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [ferramenta_imposto_renda, consulta_artigos_tool],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [42]:
agent = AgentRunner(agent_worker)
response = agent.chat("Me retorne artigos sobre LangChain na eeducação")

Added user message to memory: Me retorne artigos sobre LangChain na eeducação
=== Calling Function ===
Calling function: consulta_artigos with args: {"titulo": "LangChain na educa\u00e7\u00e3o"}


/var/folders/5z/bv195b9d0f11m9c66rw80j3r0000gp/T/ipykernel_6535/3605877752.py:15: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for artigo in busca.results()


=== Function Output ===
Título: Development and Testing of Retrieval Augmented Generation in Large Language Models -- A Case Study Report
Categoria: cs.CL
Link: http://arxiv.org/abs/2402.01733v1


Título: From Prompt Injections to SQL Injection Attacks: How Protected is Your LLM-Integrated Web Application?
Categoria: cs.CR
Link: http://arxiv.org/abs/2308.01990v4


Título: Automating Customer Service using LangChain: Building custom open-source GPT Chatbot for organizations
Categoria: cs.CL
Link: http://arxiv.org/abs/2310.05421v1


Título: Poisoned LangChain: Jailbreak LLMs by LangChain
Categoria: cs.CL
Link: http://arxiv.org/abs/2406.18122v1


Título: Breast Ultrasound Report Generation using LangChain
Categoria: eess.IV
Link: http://arxiv.org/abs/2312.03013v1

=== Calling Function ===
Calling function: consulta_artigos with args: {"titulo": "LangChain e educa\u00e7\u00e3o"}
=== Function Output ===
Título: From Prompt Injections to SQL Injection Attacks: How Protected is Your LLM-Integ

### Usando o Tavily

In [44]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


In [45]:
tavily_key = os.environ.get("TAVILY_API_KEY")

In [47]:
from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=tavily_key
)

In [49]:
tavily_tool_list = tavily_tool.to_tool_list()
for tool in tavily_tool_list:
    print(tool.metadata.name)
    

search


In [51]:
tavily_tool.search("Me rteorne artigos científicos sobre LangChain", max_results=3)

[Document(id_='700faef2-bc7e-4be1-8d7a-31894b828314', embedding=None, metadata={'url': 'https://community.revelo.com.br/faca-perguntas-ao-seu-pdf-usando-langchain-llama-2-e-python/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Durante 2023, a Meta anunciou o LLaMA 2 (de agora em diante vou chamá-lo simplesmente de Llama 2), um LLM de código aberto que é a evolução de seu modelo anterior (LLaMA 1), que pode ser usado para criar aplicações para fins comerciais. Para carregar o modelo Llama 2, como aconteceu com o carregamento de PDF e Pinecode, LangChain também nos fornece uma interface (que fácil!). Isto não significa que devolva dois parágrafos ou duas palavras, mas tem a ver com os textos que o modelo considera significativos para fornecer uma resposta: e um documento pode conter vários parágrafos. O que se vê neste artigo é 